<목차>
- multiprocessing — 프로세스 기반 병렬 처리
- Process 클래스
- 프로세스를 시작하는 3가지 방법

#### multiprocessing — 프로세스 기반 병렬 처리
- multiprocessing 은 threading 모듈과 유사한 API를 사용하여 프로세스 스포닝(spawning)을 지원하는 패키지
- multiprocessing 패키지는 지역(local)과 원격 동시성(remote concurrency)을 모두 제공하며 스레드 대신 서브 프로세스를 사용하여 전역 인터프리터 록 을 효과적으로 피한다.
- 유닉스와 윈도우에서 모두 실행된다.
  
- multiprocessing 모듈을 threading 모듈에 없는 API도 제공한다. 대표적인 예가 Pool 객체.
- Pool 객체는 여러 입력 값에 걸쳐 함수의 실행을 병렬 처리하고, 입력 데이터를 프로세스에 분산시키는 편리한 방법을 제공

In [1]:
from multiprocessing import Pool

def f(x):
    return x ** 2

if __name__ == '__main__':
    with Pool(5) as p:
        print(p.map(f, [1, 2, 3]))

#### Process 클래스
- multiprocessing에서 프로세스는 Process 객체를 생성한 후 start() 메서드를 호출해서 spawn 한다.
- Process는 threading.Thread의 API를 따른다.
- 이하 다중 프로세스 프로그램 간단 예

In [2]:
from multiprocessing import Process

def f(name):
    print('hello', name)

if __name__ == '__main__':
    p = Process(target=f, args=('bob,'))
    p.start()
    p.join()

- 이 과정에 참여하는 개별 프로세스의 ID를 보기 위해, 아래 처럼 예제를 확장한다

In [ ]:
from multiprocessing import Process
import os

def info(title):
    print(title)
    print('module name:', __name__)
    print('parent process:', os.getpid())
    print('process id:', os.getpid())

def f(name):
    info('funtion f')
    print('hello', name)

if __name__ == '__main__':
    info('main line')
    p = Process(target=f, args=('bob',))
    p.start()
    p.join()

#### 프로세스를 시작하는 3가지 방법
- Spawn: 상위 프로세스는 새로운 Python 인터프리터 프로세스를 시작한다. 하위 프로세스는 프로세스 객체의 run() 메소드를 실행하는데 필요한 리소스만 상속.
  - 특히, 상위 프로세스의 불필요한 파일 디스크립터와 핸들은 상속되지 않는다. 이 방법을 사용하여 프로세스를 시작하는 것은 fork/forkserver를 사용하는 것에 비해 다소 느리다.
  - POSIX와 Windows 플랫폼에서 가능하다.
- fork: 부모 프로세스는 os.fork() 를 사용하여 파이썬 인터프리터를 포크 합니다. 자식 프로세스는, 시작될 때, 부모 프로세스와 실질적으로 같습니다. 
  - `부모의 모든 자원이 자식 프로세스에 의해 상속`됩니다. 다중 스레드 프로세스를 안전하게 포크 하기 어렵다는 점에 주의하십시오.
  - POSIX 플랫폼에서 가능하다.
  - [3.12 버전] 파이썬이 프로세스에 여러 스레드가 있음을 감지할 수 있는 경우 이 시작 메소드가 내부적으로 호출하는 os.fork() 함수는 Deprecation Warning을 발생시킨다.
- forkserver: 프로그램이 시작되고, 포크서버 시작 방법을 선택하면, 서버 프로세스가 생성된다(spawned). 그 이후에는 새 프로세스가 필요할 때 마다 상위 프로세스가 서버에 연결하여 새 프로새스를 fork 하도록 요청한다.
  - 시스템 라이브러리나 미리 로드된 가져오기가 부작용으로 스레드를 생성하지 않는 한 포크 서버 프로세스는 단일 스레드이므로 일반적으로 os.fork()를 사용하는 것이 안전합니다. 불필요한 리소스는 상속되지 않습니다.

- POSIX에서 generate 또는 forkserver start() 메소드를 사용하면, 메소드는 자원 추적기(resource tracker) 프로세스도 같이 시작한다. 연결되지 않은 명명된(?) 시스템 자원(e.g. named semaphores or SharedMemory 객체)을 추적한다.
- 모든 프로세스가 종료되면, 리소스 추적기는 나머지 추적 객체의 연결을 해제 한다.
- 일반적으로 아무 것도 없어야 하지만, 프로세스가 signal에 의해 종료된 경우, 일부 누수(leak) 자원이 있을 수 있다.
- `누수된 세마포어나 공유 메모리 segment는 다음 재부팅 시 까지 자동으로 연결 해제되지 않는다.`
- 시스템은 제한된 수의 명명된 세마포어(named semaphore)만 허용하고, 공유 메모리 세그먼트가 주 메모리 공간 일부를 차지 하기 때문에 두 객체 모두에 문제가 된다.

8가지 종류가 있다.
- Pool.apply()
- Pool.apply_async()
- Pool.map()
- Pool.map_async()
- Pool.imap()
- Pool.imap_unordered()
- Pool.starmap()
- Pool.starmap_async()

#### Pool 사용 예시

In [ ]:
import time, os
from multiprocessing import Pool

def work_func(x):
    print("work_func:", x, "PID", os.getpid())
    time.sleep(1)
    return x ** 5

if __name__ == "__main__":
    start = int(time.time())
    cpu = 4
    pool = Pool(cpu)
    print(pool.map(work_func, range(0, 12)))

    print("***run time(sec) :", int(time.time()) - start)

#### map, async_result, imap 사용 예시
- map: 

In [ ]:
import multiprocessing as mp
import time
import random
import sys

def calculate(func, args):
    result = func(*args)
    return '%s says that %s%s = %s' % (
        mp.current_process().name, func.__name__, args, result
        )

def calculatestar(args):
    '''
    args -> (mul, (i, 7))
    이렇게 튜플로 받아서 풀어서 전달하거나,
    아래처럼 함수 내부에서 풀어서 전달한다.
    '''
    #print(*args)
    return calculate(*args) # *args(arguments) : list of arguments 즉, [mul, (i, 7)]

def calculate_rev(args):
    func, arg = args[0], args[1]
    res = func(*arg) # arg가 (0, 1) 의 튜플이기 때문에 (a, b) 인자를 받는 함수에 전달하려면 풀어서 전달해야 한다.
    return '%s says that %s%s = %s' % (mp.current_process().name, func.__name__, args, res)

def mul(a, b):
    time.sleep(0.5 * random.random())
    return a * b

def plus(a, b):
    time.sleep(0.5 * random.random())
    return a + b

if __name__ == '__main__':
    cpu = 4
    num_of_tasks = 10

    with mp.Pool(cpu) as pool:
        TASKS = [(mul, (i, 7)) for i in range(num_of_tasks)] + [(plus, (i, 8)) for i in range(num_of_tasks)] 
        '''
        [ (mul, (0, 7)), (mul, (1, 7)), ..., (mul, (9, 7)), (plut, (0, 8)), ..., (plus, (9, 8)) ]
        '''
        #results = pool.map(calculatestar, TASKS)
        map_result = pool.map(calculate_rev, TASKS) # TASK에는 일괄 적용할 인자가 들어간다!
        async_result = [pool.apply_async(calculate, t) for t in TASKS]
        imap_result = pool.imap(calculate_rev, TASKS)
        imap_unordered_result = pool.imap_unordered(calculatestar, TASKS)

        print('Ordered results - map():')
        for r in map_result:
            print('\t', r)
        print()

        print('Ordered async_results - apply_async():')
        for r in async_result:
            print('\t', r.get())
        print()

        print('Ordered results - imap():')
        for x in imap_result:
            print('\t', x)
        print()

        print('Unordered results - imap_unordered():')
        for x in imap_unordered_result:
            print('\t', x)
        print()

### GIL: global interpreter lock (전역 인터프리터 록) 이란
- 한 번에 오직 하나의 스레드가 파이썬 바이트 코드 를 실행하도록 보장하기 위해 CPython 인터프리터가 사용하는 메커니즘
- 객체 모델이 묵시적으로 동시 엑세스에 대해 안전하도록 만들어서, CPython 구현을 단순하게 만든다.
- 인터프리터 전체를 잠그는 것은 인터프리터를 다중스레드화 하기 쉽게 만드는 대신, 다중 프로세서 기계가 제공하는 병렬성의 많은 부분을 희생한다.


- 출처: https://docs.python.org/ko/3/glossary.html#term-global-interpreter-lock

### 바이트 코드란?
- 파이썬 소스 코드는 바이트 코드로 컴파일되는데, CPython 인터프리터에서 파이썬 프로그램의 내부 표현이다.
- 바이트 코드는 .pyc 파일에 캐시되어 같은 파일을 두번째 실행할 때 더 빨라지게 한다.                                              

### 예시 코드

현재 파이썬 스크립트를 실행하는 프로세스는 '메인 프로세스' 이며 name과 pid를 얻을 수 있다.

In [3]:
import multiprocessing as mp

if __name__ == "__main__":
    proc = mp.current_process()
    print(proc.name)
    print(proc.pid)


MainProcess
54972


프로세스 Spawning
- Spawning은 부모 프로세스가 OS가 요청하여 자식 프로세스를 만들어 내는 과정입니다.

In [7]:
import multiprocessing as mp
import time

def worker():
    proc = mp.current_process()
    print(proc.name)
    print(proc.pid)
    time.sleep(5)
    print("SubProcess End")

if __name__ == "__main__":
    # main process
    proc = mp.current_process()
    print(proc.name)
    print(proc.pid)
    
    # process spawning
    p = mp.Process(name="SubProcess", target=worker)
    p.start()

    print("MainProcess End")

"""
MainProcess
59204
MainProcess End
SubProcess
18956
SubProcess End
"""

MainProcess
54972
MainProcess End
